In [1]:
from modeling import EncoderExtractor, __feature_shape__
from run_training_encoders import __model_file_pattern__, __feature_data__, __label__, split_train_test_valid, normalize_feature_data
import pandas as pd
import os

In [2]:
extractor = EncoderExtractor()

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
extractors = []
best_encoders = pd.read_csv("best_encoders_new.csv")
for fold in range(1,11):
    fold_best_encoders = best_encoders[best_encoders['fold'] == fold]
    filepaths = []
    for feature in __feature_shape__.keys():
        kfold_file, _, _, l2_rate, dropout, _, _ = fold_best_encoders[fold_best_encoders['feature'] == feature].iloc[0]
        filepath = os.path.join(kfold_file[:-4] + "_models", __model_file_pattern__.format(feature, l2_rate, 0.5, fold))
        filepaths.append(filepath)
    extractor = EncoderExtractor()
    extractor.load_encoders(filepaths)
    extractors.append(extractor)

Instructions for updating:
Colocations handled automatically by placer.
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-image-l2rate0.001-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-vein-l2rate1e-05-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-xyprojection-l2rate0.01-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-color-l2rate0.01-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-texture-l2rate1e-05-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-fourier-l2rate0.1-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-shape-l2rate0.001-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-image-l2rate1e-05-dropout0.5-fold2.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-vein-l2rate1e-05-dropout0.5-fold2.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-xyprojectio

In [3]:
import numpy as np
X = [np.load(file) for file in __feature_data__.values()]
y = np.load(__label__)

In [4]:
kfold = pd.read_csv(kfold_file)

In [22]:
encoders_representations[0][0] == encoders_representations[1][0]

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [8]:
ys = []

In [15]:
(X_train, y_train), (X_valid, y_valid), (X_test, y_test) = split_train_test_valid(list(__feature_data__.keys()), kfold, 2, X, y)

In [17]:
X_train[0].shape, y_train.shape

((1525, 300, 300, 3), (1525,))

In [9]:

encoders_representations = []
ys = []
for fold, extractor in zip(range(1,11), extractors):
    (X_train, y_train), (X_valid, y_valid), (X_test, y_test) = split_train_test_valid(list(__feature_data__.keys()), kfold, fold, X, y)
    
    X_train_encoders_representation = extractor.extract(X_train)
    X_valid_encoders_representation = extractor.extract(X_valid)
    X_test_encoders_representation = extractor.extract(X_test)
    encoders_representations.append([X_train_encoders_representation, X_valid_encoders_representation, X_test_encoders_representation])
    ys.append([y_train, y_valid, y_test])

In [25]:
from sklearn.svm import SVC


valid_accuracies, test_accuracies = [], []
for fold in range(1,11):
    X_train, X_valid, X_test = encoders_representations[fold]
    y_train, y_valid, y_test = ys[fold-1]
    X_train, X_valid, X_test = normalize_feature_data("combine", X_train, X_valid, X_test)
    
    best_val = 0.0
    
    for C in [1e-1, 1,10,100,1000,10000]:
        for gamma in [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]:
            stop = False
            svm = SVC(C=C, gamma=gamma)
            svm.fit(X_train, y_train)
            val_acc = np.mean(y_valid == svm.predict(X_valid))
            test_acc = np.mean(y_test == svm.predict(X_test)) 
            print("Fold {} - C {}, gamma {} - val_acc {} - test_acc {}".format(fold, C, gamma, val_acc, test_acc))
            
            if val_acc >= best_val:
                best_val = val_acc
                best_C = C
                best_gamma = gamma
                best_test = test_acc
    print("Fold {} - best_C {} - best_gamma {} - best val acc {} - test acc {}".format(fold, best_C, best_gamma, best_val, best_test))
    valid_accuracies.append(best_val)
    test_accuracies.append(best_test)
print("Valid accuracy: {} +- {}".format(np.mean(valid_accuracies), np.std(valid_accuracies)))
print("Test accuracy: {} +- {}".format(np.mean(test_accuracies), np.std(test_accuracies)))

Fold 1 - C 0.1, gamma 1e-05 - val_acc 0.03684210526315789 - test_acc 0.03664921465968586
Fold 1 - C 0.1, gamma 5e-05 - val_acc 0.07368421052631578 - test_acc 0.07329842931937172
Fold 1 - C 0.1, gamma 0.0001 - val_acc 0.18421052631578946 - test_acc 0.16230366492146597
Fold 1 - C 0.1, gamma 0.0005 - val_acc 0.9736842105263158 - test_acc 0.9685863874345549
Fold 1 - C 0.1, gamma 0.001 - val_acc 1.0 - test_acc 0.9790575916230366
Fold 1 - C 0.1, gamma 0.005 - val_acc 0.7631578947368421 - test_acc 0.7958115183246073
Fold 1 - C 0.1, gamma 0.01 - val_acc 0.18947368421052632 - test_acc 0.23036649214659685
Fold 1 - C 1, gamma 1e-05 - val_acc 0.2 - test_acc 0.18848167539267016
Fold 1 - C 1, gamma 5e-05 - val_acc 0.9947368421052631 - test_acc 0.9842931937172775
Fold 1 - C 1, gamma 0.0001 - val_acc 1.0 - test_acc 0.9842931937172775
Fold 1 - C 1, gamma 0.0005 - val_acc 1.0 - test_acc 0.9895287958115183
Fold 1 - C 1, gamma 0.001 - val_acc 1.0 - test_acc 0.9895287958115183
Fold 1 - C 1, gamma 0.005 - v

In [12]:
X_train.shape

(1526, 700)

In [14]:
y_train.shape

(1525,)